In [2]:
using Pkg
Pkg.add("MLDatasets")

  Updating registry at `C:\Users\khjhs\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed GZip ──────────── v0.5.1
 Installed Blosc ─────────── v0.6.0
 Installed HDF5 ──────────── v0.12.5
 Installed BufferedStreams ─ v1.0.0
 Installed DataDeps ──────── v0.7.2
 Installed MLDatasets ────── v0.5.2
 Installed HTTP ──────────── v0.8.14
 Installed MAT ───────────── v0.7.0
 Installed IniFile ───────── v0.5.0
 Installed CMakeWrapper ──── v0.2.3
 Installed CMake ─────────── v1.2.0
  Updating `C:\Users\khjhs\.julia\environments\v1.2\Project.toml`
  [eb30cadb] + MLDatasets v0.5.2
  Updating `C:\Users\khjhs\.julia\environments\v1.2\Manifest.toml`
  [a74b3585] + Blosc v0.6.0
  [e1450e63] + BufferedStreams v1.0.0
  [631607c0] + CMake v1.2.0
  [d5fb7624] + CMakeWrapper v0.2.3
  [124859b0] + DataDeps v0.7.2
  [92fee26a] + GZip v0.5.1
  [f67ccb44] + HDF5 v0.12.5
  [cd3eb016] + HTTP v0.8.14
  [83e8ac13] + IniFile v0.5

InterruptException: InterruptException:

# Julia 機器學習：LIBSVM 支撐向量機

## 作業 029：

請使用 SVM 來建立一個預測模型，來對手寫數字辨識資料集，建立一個分類模型，預測數字為何。

本作業需要使用到的套件有 MLDatasets，請在執行以下範例前先安裝。

```
] add MLDatasets
```

In [5]:
Pkg.build("HDF5")

  Building CMake → `C:\Users\khjhs\.julia\packages\CMake\ULbyn\deps\build.log`
  Building Blosc → `C:\Users\khjhs\.julia\packages\Blosc\ZgcAM\deps\build.log`
  Building HDF5 ─→ `C:\Users\khjhs\.julia\packages\HDF5\Zh9on\deps\build.log`


false

In [6]:
using LIBSVM, MLDatasets, Statistics

┌ Info: Precompiling MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1242


## 讀取資料

呼叫 MNIST 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。

In [7]:
train_X, train_y = MNIST.traindata(1:10000)
test_X,  test_y  = MNIST.testdata(1:1000)

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyt

(FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N

## 資料轉換

In [8]:
train_X = Matrix(reshape(train_X, 28*28, :)')# convert to the shape that fits model
test_X = Matrix(reshape(test_X, 28*28, :)')# convert to the shape that fits model

1000×784 Array{N0f8,2} with eltype FixedPointNumbers.Normed{UInt8,8}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0

In [9]:
train_y = ["$x" for x in train_y] # convert to string type
test_y = ["$x" for x in test_y] # convert to string type

1000-element Array{String,1}:
 "7"
 "2"
 "1"
 "0"
 "4"
 "1"
 "4"
 "9"
 "5"
 "9"
 "0"
 "6"
 "9"
 ⋮  
 "1"
 "6"
 "2"
 "3"
 "9"
 "0"
 "1"
 "2"
 "2"
 "0"
 "8"
 "9"

## 預測

In [10]:
ŷ = String[]
for i = 1:size(test_X, 1)
    y_ = LIBSVM.predict(model, Matrix(test_X[i, :]'))
    append!(ŷ, y_) # 將 y_ 加入 ŷ 
end

UndefVarError: UndefVarError: model not defined

## 評估

In [13]:
accuracy(ŷ, y) = mean(ŷ .== y)


accuracy (generic function with 1 method)

In [14]:
accuracy(ŷ, test_y)

DimensionMismatch: DimensionMismatch("arrays could not be broadcast to a common size")